In [1]:
import os
import sys
import subprocess
# sys.path.insert(1, f"{subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')}/logistics/tools")
TOOLS_PATH = f"{os.getenv('MASTER_PROJECT_ROOT_FOLDER')}/logistics"
sys.path.insert(0, TOOLS_PATH)
from tools import run_tools, endf_tools
import numpy as np
# import openmc
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
# from tools import image_to_thesis
# image_to_thesis.pull_from_thesis()


In [6]:
TEMPERATURE = 900 # K
N_ITERATIONS = 1000

# for MT_INT in [2, 16, 22, 28, 51, 52, 102, 103, 104, 105, 107]:
for MT_INT in [2]:
    MT_STRING = f"{MT_INT:03d}"

    res_folder = f"results/MT{MT_INT}"
    os.system(f"rm -rf {res_folder}")
    os.makedirs(res_folder, exist_ok=True)

    df_v1 = pd.DataFrame(columns=["energy", "xs"])
    df_v3 = pd.DataFrame(columns=["energy", "xs"])

    for VERSION in ["v1", "v3"]:
        print(f"{VERSION} - MT{MT_INT}")
        if VERSION == "v1":
            sampled_h5_files_path = "/home/fne23_stjarnholm/nuclear_data/sandy_samples_v1/hdf5/F19"
        elif VERSION == "v3":
            sampled_h5_files_path = "/home/fne23_stjarnholm/nuclear_data/sandy_samples_v3"

        

        x_logspace = np.logspace(np.log10(1e-5), np.log10(2e7), 1000)

        for i in tqdm(range(N_ITERATIONS)):
            h5_filename = f"{sampled_h5_files_path}/F19-{i+1}.h5"

            h5file = h5py.File(h5_filename, 'r')
            energy_group = h5file[f'F19/energy/{TEMPERATURE}K']
            reactions_group = h5file[f'F19/reactions']

            xs_group = reactions_group[f'reaction_{MT_STRING}/{TEMPERATURE}K/xs']

            # Extract the cross section and energy data
            xs = xs_group[:]
            if len(xs) != len(energy_group[:]):
                energy = energy_group[-len(xs):]
            else:
                energy = energy_group[:]
        
            # Interpolate the cross section to the desired energy grid
            xs = np.interp(x_logspace, energy, xs)
            energy = x_logspace

            if VERSION == "v1":
                df_v1 = pd.concat([df_v1, pd.DataFrame({"energy": energy, "xs": xs})], ignore_index=True)
            elif VERSION == "v3":
                df_v3 = pd.concat([df_v3, pd.DataFrame({"energy": energy, "xs": xs})], ignore_index=True)

v1 - MT2


100%|██████████| 1000/1000 [00:04<00:00, 228.27it/s]


v3 - MT2


100%|██████████| 1000/1000 [00:04<00:00, 207.70it/s]

len(df_v1)=1000000
len(df_v3)=1000000
